In [43]:
!python --version

3413.88s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Python 3.11.5


In [44]:
import pandas as pd
import numpy as np
import trimesh
import networkx as nx

In [45]:
#stl files id
n1 = 1
n2 = 50

Reading .csv file

In [46]:
df = pd.read_csv('prop.csv')
df

,id,density,young_modulus,poisson_ratio,shear_modulus,yield_stress,ultimate_stress,strain_stress_values
0,128606,1175.949720,7.141057e+08,0.264721,4.772817e+08,2.529931e+07,2.709164e+07,"[[0.0, 0.0], [0.1, 14.477105654130623], [0.2, ..."
1,128609,879.303004,5.063934e+08,0.271033,4.959339e+08,3.208647e+07,3.561526e+07,"[[0.0, 0.0], [0.1, 11.011819136893074], [0.2, ..."
2,128610,902.743485,1.411052e+09,0.253248,5.414799e+08,1.820441e+07,2.493981e+07,"[[0.0, 0.0], [0.1, 27.737353950665053], [0.2, ..."
3,128615,1131.356417,6.882834e+08,0.206567,6.360147e+08,2.039317e+07,2.194723e+07,"[[0.0, 0.0], [0.1, 11.885238927116848], [0.2, ..."
4,128616,1152.571264,2.349322e+09,0.167839,5.214505e+08,3.872026e+07,4.511207e+07,"[[0.0, 0.0], [0.1, 25.371301361034796], [0.2, ..."
...,...,...,...,...,...,...,...,...
61761,18364,818.116707,5.814251e+08,0.346922,5.145059e+08,2.923939e+08,3.501199e+08,"[[0.0, 0.0], [0.1, 19.519911059025688], [0.2, ..."
61762,18365,660.745152,1.168510e+09,0.354228,4.184006e+08,4.151324e+08,6.013479e+08,"[[0.0, 0.0], [0.1, 31.95827171615424], [0.2, 6..."
61763,18366,711.182186,6.564377e+08,0.172712,9.807482e+07,2.463443e+08,3.270593e+08,"[[0.0, 0.0], [0.1, 20.547887959418315], [0.2, ..."
61764,18367,624.650313,8.486915e+08,0.230972,1.695812e+08,2.302253e+08,2.922557e+08,"[[0.0, 0.0], [0.1, 20.95423153650639], [0.2, 4..."


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61766 entries, 0 to 61765
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    61766 non-null  int64  
 1   density               61766 non-null  float64
 2   young_modulus         61766 non-null  float64
 3   poisson_ratio         61766 non-null  float64
 4   shear_modulus         61766 non-null  float64
 5   yield_stress          61766 non-null  float64
 6   ultimate_stress       61766 non-null  float64
 7   strain_stress_values  61766 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 3.8+ MB


parcing of.stl, features and target

In [48]:
n1 = 1
n2 = 100#1501

In [49]:
feature = []
target = []

for n in range(n1, n2):
    for i in range (1, len(df)):
        if n == df['id'][i]:
            k = i
            #print(n)
            stl_file = trimesh.load('{}.stl'.format(n))
            
            vertices_matrix = stl_file.vertices
            
            vertices_matrix_z = np.delete(vertices_matrix, 0, axis = 1)
            vertices_matrix_z = np.delete(vertices_matrix_z, 0, axis = 1)

            vertices_matrix_y = np.delete(vertices_matrix, 0, axis = 1)
            vertices_matrix_y = np.delete(vertices_matrix_y, 1, axis = 1)
            
            vertices_matrix_x = np.delete(vertices_matrix, 1, axis = 1)
            vertices_matrix_x = np.delete(vertices_matrix_x, 1, axis = 1)

            vert_x = []
            vert_y = []
            vert_z = []

            for j in range (1,len(vert_x)):
                if (vertices_matrix_x[j]>0)&(vertices_matrix_y[j]>0)&(vertices_matrix_z[j]>0)&(vertices_matrix_x[j]<5)&(vertices_matrix_y[j]<5)&(vertices_matrix_z[j]<5):
                    vert_x = np.append(vert_x, vertices_matrix_x[j])
                    vert_y = np.append(vert_y, vertices_matrix_y[j])
                    vert_z = np.append(vert_z, vertices_matrix_z[j])
            
            vert_x = np.resize(vert_x, (100,))
            vert_y = np.resize(vert_y, (100,))
            vert_z = np.resize(vert_z, (100,))
            
            feature_ = np.stack((vert_x, vert_y, vert_z), axis=1)
            feature.append(feature_)
            
            s = df['strain_stress_values'][k]
            s = s[1:-1]
            s = s.replace('[', '').replace(']', '')
            
            matrix = np.fromstring(s, dtype=float, sep=',')
            matrix = matrix.reshape(-1, 2)
            matrix = np.delete(matrix, 0, axis=1)
            matrix = np.append(matrix, np.zeros(100 - len(matrix)))
          
            target.append(matrix)
 

DNN

In [50]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [51]:
train_f, test_f, train_t, test_t = train_test_split(feature, target, test_size=0.2, random_state=123)

In [57]:
model = Sequential()
model.add(Dense(150, activation='relu', kernel_initializer='he_normal', input_dim=3))
model.add(Dense(150, activation='relu', kernel_initializer='he_normal'))
#model.add(BatchNormalization())
model.add(Dense(1, activation='linear',  kernel_initializer='he_normal'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [59]:
model.fit(train_f, train_t, epochs=50, batch_size=48, verbose=1)#, validation_data=(test_f, test_t))
model.summary()

Epoch 1/50


ValueError: in user code:

    File "/opt/anaconda3/envs/tf/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/tf/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/tf/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/tf/lib/python3.11/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/envs/tf/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/envs/tf/lib/python3.11/site-packages/keras/engine/input_spec.py", line 219, in assert_input_compatibili